# Package loading

In [1]:
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation, add, concatenate

In [2]:
tf.version

<module 'tensorflow._api.v2.version' from '/home/tao/.conda/envs/test/lib/python3.8/site-packages/tensorflow/_api/v2/version/__init__.py'>

# Datasets Manipulations

In [3]:
import os
from glob import glob
import numpy as np
from osgeo import gdal
from matplotlib import pyplot as plt

In [4]:
path = u'../../'
width = 256
num = 250

In [5]:
def load_data(path):
    images_path = glob(os.path.join(path, "tiles/*"))
    masks_path = glob(os.path.join(path, "masks/*"))
    return images_path, masks_path

In [6]:
def get_raster(raster_path):
    ds = gdal.Open(raster_path)
    data = np.empty((ds.RasterYSize, ds.RasterXSize, ds.RasterCount), dtype=np.float32)
    for b in range(1, ds.RasterCount + 1):
        band = ds.GetRasterBand(b).ReadAsArray()
        data[:, :, b-1] = band
    if data.shape[-1] > 1:
        data = norma_data(data, norma_methods='min-max')   
    return data

In [7]:
def norma_data(data, norma_methods="z-score"):
    arr = np.empty(data.shape, dtype=np.float32)
    for i in range(data.shape[-1]):
        array = data.transpose(2, 0, 1)[i, :, :]
        mins, maxs, mean, std= np.percentile(array, 1), np.percentile(array, 99), np.mean(array), np.std(array)
        if norma_methods == "z-score":
            new_array = (array-mean)/std
        else:
            new_array = np.clip(2*(array-mins)/(maxs-mins), 0, 1)
        arr[:, :, i] = new_array
    return arr

In [8]:
def pare_fun(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
    
        x = get_raster(x)
        y = get_raster(y)
        return x, y
    
    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    image.set_shape([1000, 1000, 7])
    mask.set_shape([1000, 1000, 1])
    return image, mask

In [9]:
def dataset_generator(path, width, num):
    images_path, masks_path = load_data(path)
    datasets = tf.data.Dataset.from_tensor_slices((images_path, masks_path))
    ds = datasets.map(pare_fun)
    for images, masks in ds:
        for i in range(num):
            location = np.random.randint(width/2 - 1, 999 - width/2, (2,))
            h, w = location[0], location[1]
            d1 = int(width/2 - 1)
            d2 = int(width - d1)
            image_patch = images[h-d1: h+d2, w-d1: w+d2, :]
            mask_patch = masks[h-d1: h+d2, w-d1: w+d2]
            yield image_patch, mask_patch

In [10]:
dataset_generator = dataset_generator(path, width, num)

In [93]:
for image_patch, mask_patch in dataset_generator:
    print(images_patch.shape, masks_patch.shape)

In [ ]:
# plt.imshow(first_sample['mask'][:, :])
# plt.show()

In [ ]:
# plt.imshow(first_sample['image'][:, :, 0:3])
# plt.show()

# Model Construction

In [11]:
def res_block(x, nb_filters, strides):
    res_path = BatchNormalization()(x)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same', strides=strides[0])(res_path)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same', strides=strides[1])(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1), strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)

    res_path = add([shortcut, res_path])
    return res_path

In [12]:
def encoder(x):
    to_decoder = []

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(x)
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1))(main_path)

    shortcut = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1))(x)
    shortcut = BatchNormalization()(shortcut)

    main_path = add([shortcut, main_path])
    # first branching to decoder
    to_decoder.append(main_path)

    main_path = res_block(main_path, [128, 128], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    main_path = res_block(main_path, [256, 256], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    return to_decoder

In [13]:
def decoder(x, from_encoder):
    main_path = UpSampling2D(size=(2, 2))(x)
    main_path = concatenate([main_path, from_encoder[2]], axis=3)
    main_path = res_block(main_path, [256, 256], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[1]], axis=3)
    main_path = res_block(main_path, [128, 128], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = concatenate([main_path, from_encoder[0]], axis=3)
    main_path = res_block(main_path, [64, 64], [(1, 1), (1, 1)])

    return main_path

In [14]:
def build_res_unet(input_shape):
    inputs = Input(shape=input_shape)

    to_decoder = encoder(inputs)

    path = res_block(to_decoder[2], [512, 512], [(2, 2), (1, 1)])

    path = decoder(path, from_encoder=to_decoder)

    path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path)

    return Model(inputs=inputs, outputs=path)

In [15]:
input_shape = (256, 256, 7)

In [16]:
model = build_res_unet(input_shape=input_shape)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 7) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 4096        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 512         input_1[0][0]                    
______________________________________________________________________________________________

# Model Compile

In [ ]:
# Optimizer, Loss and metrics
# Model.compile()

In [22]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [88]:
def dice_loss(y_true, y_pred): 
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - (numerator / denominator)

In [97]:
def cedice_loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)
        return 1 - numerator / denominator
    y_true = tf.cast(y_true, tf.float32)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred) + dice_loss(y_true, y_pred)
    return tf.reduce_mean(loss)

In [90]:
y_pred = np.array([0.5, 0.6, 0.2, 0.1], dtype=np.float32)
y_true = np.array([1, 1, 0, 0], dtype=np.float32)

In [91]:
y_pred = y_pred.reshape((2, 2))
y_true = y_true.reshape((2, 2))

In [92]:
a = tf.math.sigmoid(y_pred)
a

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.62245935, 0.6456563 ],
       [0.54983395, 0.5249792 ]], dtype=float32)>

In [93]:
o = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred)
o

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.474077  , 0.43748793],
       [0.79813886, 0.7443967 ]], dtype=float32)>

In [99]:
loss = cedice_loss(y_true, tf.math.log(y_pred / (1 - y_pred)))
loss

<tf.Tensor: shape=(), dtype=float32, numpy=2.0714836>

In [98]:
loss1 = cedice_loss(y_true, y_pred)
loss1

<tf.Tensor: shape=(), dtype=float32, numpy=0.9664663>

In [96]:
dice_loss = dice_loss(y_true, y_pred)
dice_loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.35294116>

In [103]:
model.compile(loss=dice_loss, optimizer=optimizer, metrics=['pixel_iou'])

# Model Training

In [104]:
model.fit(dataset_generator)

ValueError: in user code:

    /home/tao/.conda/envs/test/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/tao/.conda/envs/test/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/tao/.conda/envs/test/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/tao/.conda/envs/test/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/tao/.conda/envs/test/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/tao/.conda/envs/test/lib/python3.8/sit